In [10]:
%load_ext autoreload
%autoreload 2

import geopandas as gpd
import pandas as pd
import polars as pl
import polars_st as st
import pyogrio
from electoralyze import region
from electoralyze.common import constants
from electoralyze.common.geometry import to_geopandas, to_geopolars
from electoralyze.common.testing.region_fixture import (
    RegionMocked,
    create_fake_regions,
    read_true_geometry,
    read_true_metadata,
)
from electoralyze.region.redistribute import redistribute
from electoralyze.region.redistribute.mapping import (
    _create_intersection_area_mapping,
    _get_remaining_area,
    create_region_mapping_base,
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
intersection_area = _create_intersection_area_mapping(
    geometry_from=read_true_geometry("triangle"),
    geometry_to=read_true_geometry("square"),
)
intersection_area.pipe(print)

shape: (5, 3)
┌──────────┬────────┬─────────┐
│ triangle ┆ square ┆ mapping │
│ ---      ┆ ---    ┆ ---     │
│ str      ┆ str    ┆ f64     │
╞══════════╪════════╪═════════╡
│ A        ┆ main   ┆ 32.0    │
│ B        ┆ main   ┆ 16.0    │
│ C        ┆ main   ┆ 16.0    │
│ B        ┆ null   ┆ 16.0    │
│ C        ┆ null   ┆ 16.0    │
└──────────┴────────┴─────────┘


In [18]:
import shutil
import tempfile

temp_dir = tempfile.TemporaryDirectory(delete=False)
region_ = create_fake_regions(temp_dir.name)
region_
region_.quadrant.geometry
region_.triangle.geometry
region_.rectangle.geometry
region_.square.geometry
region_.l_and_r.geometry
shutil.rmtree(temp_dir.name)

/home/andre/git/private/electoralyze/packages/electoralyze/electoralyze/common/testing/region_fixture.py:260: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  region_gdf.pipe(to_geopandas).to_file(region_shape_file[region_id], driver="ESRI Shapefile")
/home/andre/git/private/electoralyze/.pixi/envs/default/lib/python3.13/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'quadrant_name' to 'quadrant_n'
  ogr_write(
/home/andre/git/private/electoralyze/packages/electoralyze/electoralyze/common/testing/region_fixture.py:260: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  region_gdf.pipe(to_geopandas).to_file(region_shape_file[region_id], driver="ESRI Shapefile")
/home/andre/git/private/electoralyze/.pixi/envs/default/lib/python3.13/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'triangle_name' to 'triangle_n'
  ogr

Loading raw...
Extracting...
Transforming...
Saving...
Done!
Loading raw...
Extracting...
Transforming...
Saving...
Done!
Loading raw...
Extracting...
Transforming...
Saving...
Done!
Loading raw...
Extracting...
Transforming...
Saving...
Done!
Loading raw...
Extracting...
Transforming...
Saving...
Done!


In [19]:
RegionMocked

electoralyze.common.testing.region_fixture.RegionMocked

In [20]:
data_by_from = pl.DataFrame({"triangle": ["A"], "data": [1]})
print(data_by_from)
redistribute(
    data_by_from,
    region_from=region_.triangle,
    region_to=region_.quadrant,
    mapping="intersection_area",
    redistribute_with_full=False,
)

shape: (1, 2)
┌──────────┬──────┐
│ triangle ┆ data │
│ ---      ┆ ---  │
│ str      ┆ i64  │
╞══════════╪══════╡
│ A        ┆ 1    │
└──────────┴──────┘


quadrant,data
str,f64
"""O""",0.375
"""P""",0.375
"""N""",0.125
"""M""",0.125


In [21]:
data_by_from = pl.DataFrame({"rectangle": ["X", "Y", "Z"], "data": [1, 1, 1]})
print(data_by_from)
redistribute(
    data_by_from,
    region_from=region_.rectangle,
    region_through=region_.quadrant,
    region_to=region_.triangle,
    mapping="intersection_area",
    redistribute_with_full=False,
)

shape: (3, 2)
┌───────────┬──────┐
│ rectangle ┆ data │
│ ---       ┆ ---  │
│ str       ┆ i64  │
╞═══════════╪══════╡
│ X         ┆ 1    │
│ Y         ┆ 1    │
│ Z         ┆ 1    │
└───────────┴──────┘


triangle,data
str,f64
"""C""",0.75
"""B""",0.75
"""A""",1.5


In [22]:
data_by_from = pl.DataFrame({"square": ["main"], "data": [100]})
print(data_by_from)
redistribute(
    data_by_from,
    region_from=region_.square,
    region_through=region_.quadrant,
    region_to=region_.l_and_r,
    mapping="intersection_area",
    redistribute_with_full=False,
)

shape: (1, 2)
┌────────┬──────┐
│ square ┆ data │
│ ---    ┆ ---  │
│ str    ┆ i64  │
╞════════╪══════╡
│ main   ┆ 100  │
└────────┴──────┘


l_and_r,data
str,f64
"""R""",50.0
"""L""",50.0


In [23]:
data_by_from = pl.DataFrame({"square": ["main"], "data": [100]})
print(data_by_from)
redistribute(
    data_by_from,
    region_from=region_.square,
    region_through=region_.triangle,
    region_to=region_.l_and_r,
    mapping="intersection_area",
    redistribute_with_full=False,
)

shape: (1, 2)
┌────────┬──────┐
│ square ┆ data │
│ ---    ┆ ---  │
│ str    ┆ i64  │
╞════════╪══════╡
│ main   ┆ 100  │
└────────┴──────┘


l_and_r,data
str,f64
null,25.0
"""R""",37.5
"""L""",37.5


In [ ]:
def test_redistribute_through(
    region_from_id: str,
    region_to_id: str,
    region_through_id: str,
    redistribute_kwargs: dict,
):
    redistribute(
        region_from=region.from_id(region_from_id),
        region_through=region.from_id(region_through_id),
        region_to=region.from_id(region_to_id),
        **redistribute_kwargs,
    )

In [26]:
data_by_from = pl.DataFrame({"l_and_r": ["L", "R"], "data": [32, 64]})
print(data_by_from)
redistribute(
    data_by_from,
    region_from=region_.l_and_r,
    region_through=region_.triangle,
    region_to=region_.quadrant,
    mapping="intersection_area",
    redistribute_with_full=False,
)

shape: (2, 2)
┌─────────┬──────┐
│ l_and_r ┆ data │
│ ---     ┆ ---  │
│ str     ┆ i64  │
╞═════════╪══════╡
│ L       ┆ 32   │
│ R       ┆ 64   │
└─────────┴──────┘


quadrant,data
str,f64
"""P""",22.0
null,24.0
"""O""",20.0
"""N""",18.0
"""M""",12.0
